In [4]:
## import the tools
import torch
import glob
from pathlib import Path
import os
import numpy as np
from sklearn.neighbors import NearestNeighbors as NN
import glob
import pandas as pd
import ipdb
import laspy
import open3d as o3d
from tqdm.notebook import tqdm
import numpy as np


import open3d as o3d
import laspy


In [5]:
### TEST DATA FILENAMES 
#VAL FILES
filename = "PUNKTSKY_00005_1km_6220_495" # OKAY EVEN THO IT IS SRAIGHT LINE
filename = "PUNKTSKY_00005_1km_6221_452" # REALLY BAD EVEN THO IT IS SRAIGHT LINE
# TEST FILES
filename = "PUNKTSKY_00005_1km_6219_494" 
filename = "PUNKTSKY_00005_1km_6211_474" 


In [12]:
# eval_file = "/home/jfriis/msc_data/eval/"+filename+".laz"
# data_val_path_file = "/home/jf/data/denmark/raw/val/"+filename+".laz"
path_to_file = "/home/jf/data/denmark/raw/test/"+filename+".laz"


In [13]:
data = laspy.read(path_to_file, laz_backend=laspy.compression.LazBackend.LazrsParallel)
all_point_data = np.stack([data.X, data.Y, data.Z], axis=0).transpose((1, 0))
print(all_point_data.shape)

4095
(15599105, 3)


In [14]:
import numpy as np

def voxelization(point_cloud, voxel_size):
    # Calculate the minimum and maximum coordinates of the point cloud
    min_coord = np.min(point_cloud, axis=0)
    max_coord = np.max(point_cloud, axis=0)

    # Calculate the number of voxels in each dimension
    voxel_counts = ((max_coord - min_coord) / voxel_size).astype(int) + 1

    # Create an empty voxel grid
    voxel_grid = np.zeros(voxel_counts, dtype=int)

    # Assign points to their respective voxels
    voxel_indices = ((point_cloud - min_coord) / voxel_size).astype(int)
    voxel_grid[tuple(voxel_indices.T)] = 1

    return voxel_grid


In [15]:
def count_voxels(voxel_grid):
    num_empty_voxels = np.count_nonzero(voxel_grid == 0)
    num_occupied_voxels = np.count_nonzero(voxel_grid == 1)
    
    return num_empty_voxels, num_occupied_voxels


In [16]:
def calculate_sparsity(voxel_grid):
    num_empty_voxels, num_occupied_voxels = count_voxels(voxel_grid)
    total_voxels = voxel_grid.size

    sparsity = num_empty_voxels / total_voxels

    return sparsity


In [17]:
# Assuming you have a point cloud represented as a numpy array
point_cloud = np.array([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]])

# Voxelization
voxel_size = 0.1
voxel_grid = voxelization(all_point_data, voxel_size)

# Calculate sparsity
sparsity = calculate_sparsity(voxel_grid)
print("Sparsity:", sparsity)


MemoryError: Unable to allocate 1.00 EiB for an array with shape (1000001, 1000001, 144231) and data type int64